# Descargar Librerias

In [ ]:
#Descargar siempre cuando usen Colab
#!pip install praw tqdm

# Iniciar el Extractor y Completar debajo

In [12]:
import praw
import pandas as pd
import os
import time
from tqdm import tqdm

# Configura tu instancia de PRAW con tus credenciales de la aplicación y un user_agent único
reddit_config = {
    'client_id': '9zK4SSfBSCMJPnpX1rbg4Q',
    'client_secret': 'CI5_-mXS5NGVZMvNjmOb6kyE60fG_A',
    'user_agent': 'MiScriptRedditBot/v1.7 (by Sheincito en Reddit)'
}

# Define el subreddit a extraer
subreddit_to_extract = input("Ingrese el nombre del subreddit: ")

# Define el filtro para los títulos de las publicaciones
filtro_titulo = input("Ingrese el filtro para los títulos de las publicaciones (deje en blanco para no filtrar por título): ")

# Define el filtro para el cuerpo de las publicaciones
filtro_cuerpo = input("Ingrese el filtro para el cuerpo de las publicaciones (deje en blanco para no filtrar por cuerpo): ")

# Nombre del archivo CSV basado en el subreddit extraído
csv_filename = f"extractor_{subreddit_to_extract}.csv"

# Función para cargar los títulos existentes del archivo CSV
def cargar_titulos_existentes(filename, subreddit_name):
    if os.path.isfile(filename):  
        try:
            df = pd.read_csv(filename)
            if df.empty:
                return df
            else:
                return set(df['Reddit'])
        except pd.errors.EmptyDataError:
            return df
    else: 
        return pd.DataFrame(columns=[subreddit_name])

# Función para recopilar comentarios
def recopilar_comentarios(subreddit_name, filename, reddit, filtro_titulo=None, filtro_cuerpo=None):
    # Cargar los títulos existentes del archivo CSV
    titulos_existentes = cargar_titulos_existentes(filename, subreddit_name)
    
    # Obtiene el subreddit
    subreddit = reddit.subreddit(subreddit_name)
    
    # Lista para almacenar los comentarios
    comentarios = []
    
    # Convertir el filtro de título a minúsculas si está presente
    filtro_titulo = filtro_titulo.lower() if filtro_titulo else None
    
    # Convertir el filtro de cuerpo a minúsculas si está presente
    filtro_cuerpo = filtro_cuerpo.lower() if filtro_cuerpo else None
    
    # Dividir el filtro de título en palabras si contiene varias palabras
    filtro_titulo_palabras = filtro_titulo.split() if filtro_titulo else None
    
    # Dividir el filtro de cuerpo en palabras si contiene varias palabras
    filtro_cuerpo_palabras = filtro_cuerpo.split() if filtro_cuerpo else None
    
    # Bucle para recopilar comentarios
    for submission in tqdm(subreddit.new(limit=None), desc='Recopilando comentarios'):
        # Verificar si la publicación cumple con al menos uno de los filtros
        if (not filtro_titulo or any(word in submission.title.lower() for word in filtro_titulo_palabras)) \
            or (not filtro_cuerpo or any(word in submission.selftext.lower() for word in filtro_cuerpo_palabras)):
            # Obtener todos los comentarios del hilo
            submission.comments.replace_more(limit=None)
            count = 0
            for comment in submission.comments.list():
                # Verificar si el comentario tiene más o igual 6 palabras
                if len(comment.body.split()) >= 6:
                    # Limitar a 15comentarios por publicación
                    if count >= 15:
                        break
                    # Agregar el comentario a la lista de comentarios
                    comentarios.append({'Reddit': submission.id, 'Comentario': comment.body})
                    count += 1
                    
                    # Esperar 1 segundo antes de procesar el siguiente comentario
                    time.sleep(1)
    
    # Convierte la lista de comentarios en un DataFrame
    df = pd.DataFrame(comentarios)
    
    # Guarda los comentarios en un archivo CSV (modo 'a' para agregar datos sin sobrescribir)
    df.to_csv(filename, index=False, mode='a', header=not os.path.exists(filename))
    
    # Mensaje de finalización
    print(f"Se han extraído y guardado exitosamente {len(comentarios)} comentarios nuevos del subreddit '{subreddit_name}'.")
    print("Proceso finalizado.")

# Configura PRAW con la configuración especificada
reddit = praw.Reddit(**reddit_config)

# Llama a la función para recopilar comentarios y guardarlos en un archivo CSV
recopilar_comentarios(subreddit_to_extract, csv_filename, reddit, filtro_titulo, filtro_cuerpo)


Ingrese el nombre del subreddit: argentina
Ingrese el filtro para los títulos de las publicaciones (deje en blanco para no filtrar por título): milei
Ingrese el filtro para el cuerpo de las publicaciones (deje en blanco para no filtrar por cuerpo): milei


Recopilando comentarios: 975it [28:16,  1.74s/it]

Se han extraído y guardado exitosamente 1559 comentarios nuevos del subreddit 'argentina'.
Proceso finalizado.


# Cargamos el DF generado para probar

In [13]:
# Carga el archivo CSV que contiene los datos del DataFrame
df = pd.read_csv(csv_filename)
pd.options.display.max_colwidth = 10000

In [26]:
df.sample(10)

,Reddit,Comentario
558,1b8cy7y,"parece joda que detengan a alguien poner comentar en la redes, si fuera por eso el presidente ya debería tener perpetua. Muy cara dura todo"
529,1b8fs2s,"***""Observamos cómo el sistema argentino de ciencia y tecnología se acerca a un peligroso precipicio y nos desalientan las consecuencias que esta situación podría tener tanto para el pueblo argentino como para el mundo""***\n\n***""Para el mundo"",*** uyyy si, esos papers del ano de Batman van a extrañarse mucho en todo el mundo. así como tantos desarrollos argentinos que hacen mover al mundo hoy en día, y que nos hace ser el país más desarrollado del mundo desde hace 30 años."
229,1bamo1g,"Y bueno que vayan a votar, los presidentes no se eligen por el tamaño de la marcha"
1177,1b3e7cc,Después se llenan la boca de amor y democracia.
324,1b9s3rg,"Me parece bueno marchar por ""derechos en peligro"" pero mira a esta mina. Su mayor preocupacion es poder comprarse un nuevo iphone, quien mierda era para decirle a la gente comun que psota sufre que hacer? Que payasa"
1498,1b15ls1,"Jaja, si viniera a salvar a alguien lo haría más al estilo de Jesús de Laferrere...jeje"
937,1b5ty1w,"Sueño que K5N vea esto y digan ""LAS POLEMICAS PALABRAS DE MEGAHIPERSUPERULTRISIMA- DERECHA"" en el congreso"
1277,1b2i0nk,"Está perfecto, porque harían documentos en un lenguaje que no existe?"
761,1b7a2g1,"Danann se une al club de los Gerardo Morales, aunque ya se sospechaba"
12,1bc7h4k,"Pero ahí arriba dice Linguahouse, que parece ser un curso a lo OpenEnglish. \n \n Porque en Oxford usarían ese material cuando tienen el propio el cual es muy bueno?"
